In [20]:
import os
import requests
from dotenv import load_dotenv

# 加載 .env 文件中的環境變量
load_dotenv()

# 獲取 API 金鑰
api_key = os.getenv('API_KEY')
url = 'https://pro-api.coinmarketcap.com/v4/dex/spot-pairs/latest?network_slug=Solana&base_asset_contract_address=2zMMhcVQEXDtdE6vsFS7S7D5oUodfJHE8vd1gnBouauv'

# 設置請求頭
headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': api_key,
}

# 發送請求
response = requests.get(url, headers=headers)

# 檢查請求是否成功
if response.status_code == 200:
    data = response.json()
    print(data)  # 輸出數據
else:
    print(f"Error: {response.status_code} - {response.text}")

{'data': [{'quote': [{'convert_id': '2781', 'price': 0.03320611781297352, 'price_by_quote_asset': 0.000163189511011608, 'last_updated': '2024-12-19T02:53:10.224Z', 'volume_24h': 315541853.4210677, 'percent_change_price_1h': -0.0074369214, 'percent_change_price_24h': 0.1028468957, 'liquidity': 23974013.385682512, 'fully_diluted_value': 2951653953.4229465}], 'scroll_id': '50', 'contract_address': 'B4Vwozy1FGtp8SELXSXydWSzavPUGnJ77DURV2k4MhUV', 'name': 'PENGU/SOL', 'base_asset_id': '33461669', 'base_asset_ucid': '34466', 'base_asset_name': 'Pudgy Penguins', 'base_asset_symbol': 'PENGU', 'base_asset_contract_address': '2zMMhcVQEXDtdE6vsFS7S7D5oUodfJHE8vd1gnBouauv', 'quote_asset_id': '22370960', 'quote_asset_ucid': '16116', 'quote_asset_name': 'Wrapped SOL', 'quote_asset_symbol': 'SOL', 'quote_asset_contract_address': 'So11111111111111111111111111111111111111112', 'dex_id': '10557', 'dex_slug': 'raydium-clmm', 'network_id': '16', 'network_slug': 'Solana', 'last_updated': '2024-12-19T02:53:1

In [43]:

import os
import requests
from dotenv import load_dotenv

# 加載 .env 文件中的環境變量
load_dotenv()

# 獲取 API 金鑰
api_key = os.getenv('API_KEY')
url = 'https://pro-api.coinmarketcap.com/v4/dex/pairs/ohlcv/historical?contract_address=B4Vwozy1FGtp8SELXSXydWSzavPUGnJ77DURV2k4MhUV&network_id=16&time_period=15m&interval=15m&count=500' 
# 設置請求頭
headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': api_key,
}

# 發送請求
response = requests.get(url, headers=headers)

# 檢查請求是否成功
if response.status_code == 200:
    data1 = response.json()
    print(data1)  # 輸出數據
else:
    print(f"Error: {response.status_code} - {response.text}")
data1


{'data': [{'quotes': [{'quote': [{'convert_id': '2781', 'open': 0.027930835411092748, 'high': 0.027930835411092748, 'low': 0.027930835411092748, 'close': 0.027930835411092748, 'last_updated': None, 'volume': 267.11886844223744}], 'time_open': '2024-12-17T12:45:00.000Z', 'time_close': '2024-12-17T12:59:59.999Z'}, {'quote': [{'convert_id': '2781', 'open': 0.027930835411092748, 'high': 0.07252444011416019, 'low': 0.03935857905176072, 'close': 0.05063610490438345, 'last_updated': None, 'volume': 86798182.45918742}], 'time_open': '2024-12-17T13:00:00.000Z', 'time_close': '2024-12-17T13:14:59.999Z'}, {'quote': [{'convert_id': '2781', 'open': 0.05063610490438345, 'high': 0.05650163492439122, 'low': 0.049367570462344385, 'close': 0.050353427610671186, 'last_updated': None, 'volume': 39320434.988092}], 'time_open': '2024-12-17T13:15:00.000Z', 'time_close': '2024-12-17T13:29:59.999Z'}, {'quote': [{'convert_id': '2781', 'open': 0.050353427610671186, 'high': 0.05057847113880873, 'low': 0.042966487

{'data': [{'quotes': [{'quote': [{'convert_id': '2781',
       'open': 0.027930835411092748,
       'high': 0.027930835411092748,
       'low': 0.027930835411092748,
       'close': 0.027930835411092748,
       'last_updated': None,
       'volume': 267.11886844223744}],
     'time_open': '2024-12-17T12:45:00.000Z',
     'time_close': '2024-12-17T12:59:59.999Z'},
    {'quote': [{'convert_id': '2781',
       'open': 0.027930835411092748,
       'high': 0.07252444011416019,
       'low': 0.03935857905176072,
       'close': 0.05063610490438345,
       'last_updated': None,
       'volume': 86798182.45918742}],
     'time_open': '2024-12-17T13:00:00.000Z',
     'time_close': '2024-12-17T13:14:59.999Z'},
    {'quote': [{'convert_id': '2781',
       'open': 0.05063610490438345,
       'high': 0.05650163492439122,
       'low': 0.049367570462344385,
       'close': 0.050353427610671186,
       'last_updated': None,
       'volume': 39320434.988092}],
     'time_open': '2024-12-17T13:15:00.00

In [44]:
import pandas as pd
quotes = data1['data'][0]['quotes']

# 將 quotes 數據轉換為 DataFrame
df = pd.DataFrame()

# 收集所有 quote 的數據
dataframes = []

for quote in quotes:
    quote_data = quote['quote'][0]  # 取出每個 quote 的數據
    quote_data['time_open'] = quote['time_open']
    quote_data['time_close'] = quote['time_close']
    dataframes.append(pd.DataFrame([quote_data]))  # 將每個 quote 轉換為 DataFrame 並添加到列表

# 合併所有 DataFrame
df = pd.concat(dataframes, ignore_index=True)
# 將 DataFrame 保存為 CSV 文件
df.to_csv('quotes_data.csv', index=False)
# 顯示 DataFrame
df



,convert_id,open,high,low,close,last_updated,volume,time_open,time_close
0,2781,0.027931,0.027931,0.027931,0.027931,None,2.671189e+02,2024-12-17T12:45:00.000Z,2024-12-17T12:59:59.999Z
1,2781,0.027931,0.072524,0.039359,0.050636,None,8.679818e+07,2024-12-17T13:00:00.000Z,2024-12-17T13:14:59.999Z
2,2781,0.050636,0.056502,0.049368,0.050353,None,3.932043e+07,2024-12-17T13:15:00.000Z,2024-12-17T13:29:59.999Z
3,2781,0.050353,0.050578,0.042966,0.049179,None,2.985489e+07,2024-12-17T13:30:00.000Z,2024-12-17T13:44:59.999Z
4,2781,0.049179,0.056618,0.048419,0.055351,None,3.181618e+07,2024-12-17T13:45:00.000Z,2024-12-17T13:59:59.999Z
...,...,...,...,...,...,...,...,...,...
149,2781,0.032951,0.033381,0.031105,0.031156,None,4.406346e+06,2024-12-19T02:00:00.000Z,2024-12-19T02:14:59.999Z
150,2781,0.031156,0.032354,0.031124,0.031834,None,3.256624e+06,2024-12-19T02:15:00.000Z,2024-12-19T02:29:59.999Z
151,2781,0.031834,0.032696,0.031632,0.032361,None,2.396206e+06,2024-12-19T02:30:00.000Z,2024-12-19T02:44:59.999Z
152,2781,0.032361,0.034081,0.032276,0.033777,None,4.235579e+06,2024-12-19T02:45:00.000Z,2024-12-19T02:59:59.999Z
